# Data Collection

In [4]:
"""
Created on Mon Apr 11 18:30:05 2016
"""
# import packages 
import pandas as pd                   # data management
import matplotlib.pyplot as plt       # graphics 
import matplotlib as mpl              # graphics parameters
import numpy as np                    # numerical calculations 


# Checking Python version 
import datetime as dt 
import sys
print('Today is', dt.date.today())
print('What version of Python are we running? \n', sys.version, sep='') 

Today is 2016-04-20
What version of Python are we running? 
3.5.1 |Anaconda 4.0.0 (64-bit)| (default, Feb 16 2016, 09:49:46) [MSC v.1900 64 bit (AMD64)]


In [2]:
#%%
from matplotlib import cm, colors, _cm

# Data Collection
import urllib
import jsonhttp://nbviewer.ipython.org/gist/hyeshik/cf9f3d7686e07eedbfda?revision=6
from time import time
import sqlite3

In [5]:
con = sqlite3.connect("C:\\Users\\Dongjin\\Desktop\\MV_02_realestate.db")
# sqlite3.connect
# connect(database[, timeout, detect_types, isolation_level,check_same_thread, factory, cached_statements, uri])
# Opens a connection to the SQLite database file *database*.

#  (City Code inserted in the DB by hand as follows)
state_info=pd.read_sql("select * from addr_si_code_tbl;",con)
state_info.head(10)

,si_code,name,update_tm
0,11,서울특별시,1460401951
1,26,부산광역시,1460401951
2,27,대구광역시,1460401951
3,28,인천광역시,1460401951
4,29,광주광역시,1460401951
5,30,대전광역시,1460401951
6,31,울산광역시,1460401951
7,36,세종특별자치시,1460401951
8,41,경기도,1460401951
9,42,강원도,1460401951


In [26]:
# Define the si, gun, gu data collection 
def Address_Info( code, type='SGG'  ):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code=%d&bjd_gbn=%s" %(code,type) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])


In [27]:
# Get apartment information 
# http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=11350105

def Apartment_Info(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=%d" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])


In [39]:
# Crawling the information
for name, code in state_info[['name','si_code']].itertuples(index=False):
    gu_info = Address_Info(code)
    values=gu_info.apply(lambda x : u"(%d, %s, '%s')" % (code, x['CODE'], x['CODE_VALUE'] ), axis=1)
    insert_sql = u"insert into addr_gu_code_tbl( si_code, gu_code, name) values %s ;" % (u",".join(values))
    con.execute(insert_sql)
    con.commit()
city_info=pd.read_sql("select * from addr_gu_code_tbl;",con)
city_info.head()


,si_code,gu_code,name
0,11,680,강남구
1,11,740,강동구
2,11,305,강북구
3,11,500,강서구
4,11,620,관악구


In [ ]:

# Crawling the detail location information
for si,gu in city_info[['si_code','gu_code']].itertuples(index=False):
    code = "%d%d" % (si,gu)
    info = Address_Info(int(code),'EMD')
    values=info.apply(lambda x : u"(%d, %d, %s, '%s')" % (si,gu, x['CODE'], x['CODE_VALUE'] ), axis=1)
    insert_sql = u"insert into addr_dong_code_tbl( si_code, gu_code, dong_code,name) values %s ;" % (u",".join(values))
    con.execute(insert_sql)
    con.commit()
    print ("%s code inserted(%d)" % (code, info.count()[0]))

In [41]:
# Collection of the "Dong-Code" - Similar to zipcode
town_info=pd.read_sql("select * from addr_dong_code_tbl;",con)
print (town_info.head())


   si_code  gu_code  dong_code name
0       11      680        103  개포동
1       11      680        108  논현동
2       11      680        106  대치동
3       11      680        118  도곡동
4       11      680        105  삼성동


In [30]:
# http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=11350105
def getAptDetailInfo(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=%s" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return data


In [31]:
def extractAptInfo(a):
    info = []
    for name in ['KAPT_CODE', 'KAPT_DONG_CNT','KAPT_USEDATE', 'KAPTD_WTIMESUB', 'SUBWAY_STATION', 'SUBWAY_LINE', 'CODE_HALL', 'KAPT_BCOMPANY' ] :
        info.append(a[u'resultMap_kapt'][name]);
    cnt = 0
    for x in a[u'resultMap_kapt_areacnt']:
        if( type(x[u'KAPTDA_CNT']) is int ):
            info.append(x[u'KAPTDA_CNT'])
        else :
            info.append(0)
#           cnt = cnt + x[u'KAPTDA_CNT']
 #   info.append(cnt)
    return info

In [ ]:
# Collecting Apartment information
for code in town_info.apply(lambda x : x['si_code']*1000000+x['gu_code']*1000+x['dong_code'], axis=1):
    print ("%d insert"% (code))
    apt = Apartment_Info(code)
    if apt.empty :
        continue
    apt.to_sql('apt_code_tbl2', con, index=False, if_exists='append' )

In [32]:
# Aparment information 
apt_info=pd.read_sql("select * from apt_code_tbl2;",con)
apt_info.head()

,BJD_CODE,ENERGY_B_COUNT,KAPT_CODE,KAPT_NAME
0,1168010300,0,A13593901,LG개포자이아파트
1,1168010300,0,A13593908,SH공사대치1단지
2,1168010300,0,A13524006,개포2차 현대아파트
3,1168010300,0,A13580602,개포경남아파트
4,1168010300,0,A13524009,개포대치2단지


In [ ]:
# Crawling Detail infromation( Year, Subway, etc)
apt_details = pd.DataFrame(columns=['KAPT_CODE', 'KAPT_DONG_CNT','KAPT_USEDATE', 'KAPTD_WTIMESUB', 'SUBWAY_STATION', 'SUBWAY_LINE', 'CODE_HALL','KAPT_BCOMPANY', 'KAPTDA_CNT1', 'KAPTDA_CNT2', 'KAPTDA_CNT3', 'KAPTDA_CNT4'])

for code in apt_info['KAPT_CODE']:
    a = getAptDetailInfo(code)
    if not bool(a):
        continue
    info = extractAptInfo(a)
    if not bool(info):
        continue
    n=len(apt_details)
    apt_details.loc[n] = info
    print ("%s apt data collection completed(%d)" % (code,n)